In [1]:
from IPython.display import HTML

HTML('''<script>
 function code_toggle() {
   if (code_shown){
     $('div.input').hide('500');
     $('#toggleButton').val('Show Code')
   } else {
     $('div.input').show('500');
     $('#toggleButton').val('Hide Code')
   }
   code_shown = !code_shown
 }

 $( document ).ready(function(){
   code_shown=false;
   $('div.input').hide()
 });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>'''
   )

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')
%output holomap='scrubber'

In [ ]:
# holovies settings
# get colors
colors = hv.core.options.Cycle.default_cycles['default_colors']

# sizes
ticks = 16
labels = 20
title = 20

# dimensions
dim_dr = hv.Dimension('dr/dt', unit='s^-2')
dim_r = hv.Dimension('r', unit='Hz')
dim_r2 = hv.Dimension('r', unit='Hz')
dim_t = hv.Dimension('Time', unit='s')
dim_re = hv.Dimension('rE', unit='Hz')
dim_ri = hv.Dimension('rI', unit='Hz')


In [ ]:
%opts Curve (line_width=4) HLine (line_width=2) {+axiswise}
%opts HLine (color='k' line_dash='dashed') Scatter (size=15)
%opts Curve [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title} yticks=4]
%opts Histogram [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title} yticks=4]
%opts Scatter [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title} yticks=4] {+axiswise}
%opts Overlay [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title} yticks=4]
%opts Image [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title}]
%opts Polygons [fontsize={'ticks':ticks, 'ylabel':labels, 'xlabel':labels,'title':title}]
%opts Overlay [show_frame=False show_grid=False bgcolor='white']
%opts Curve [show_grid=False bgcolor='white'] (linewidth=3)
%opts Histogram (facecolor='k' edgecolor='k')
%opts Scatter [show_grid=False bgcolor='white'] (color='k' s=75)
%opts Image [xaxis=None yaxis=None show_frame=False colorbar=False] (cmap='Greys' clims=(-1.5,1.5))
%opts Curve [fig_rcparams={'xtick.major.width':1,'xtick.major.size':5,'ytick.major.size':5}]
%opts Scatter [fig_rcparams={'xtick.major.width':1,'xtick.major.size':5,'ytick.major.size':5}]
%opts Overlay [fig_rcparams={'xtick.major.width':1,'xtick.major.size':5,'ytick.major.size':5}]
%opts Layout [fig_rcparams={'xtick.major.width':1,'xtick.major.size':5,'ytick.major.size':5}]
%opts Spread [fig_rcparams={'xtick.major.width':1,'xtick.major.size':5,'ytick.major.size':5}]
%opts Arrow (arrow_line_width=4 arrow_size=10)
%opts VectorField [size_index=3]
%output size=150

In [ ]:
# solver
def euler_solve(dx, x0, par, dt, nT, noise=None):
    '''Simulates the system of differential equations specified by dx, using forward Euler.
    
    Parameters
    ----------
    dx : function
        Function of form dx(x, par), which determines how the state will change given current state
    x0 : array
        Starting parameters
    par : unknown
        Whatever parametes go into dx
    dt : float
        Timestep size
    nT : int
        Number of timesteps
    noise : array, to be implemented
        If given, what noise to be added at every time step (could possibly be a function)
    
    Output
    ------
    array
        Simulated values
    '''
    # preamble
    nS = len(x0) # nr of equations/signals
    out = np.zeros((nS, nT)) # predefined output
    out[:, 0] = x0
    
    # loop over time
    for i in range(nT-1):
        out[:, i+1] = out[:, i] + dt*dx(out[:, i], par)
    
    # return
    return out

# 1D phaseplane plotter
def phaseplane_1D(dx, par, x, add_arrows=None, add_points=False):
    '''Plots the phaseplane in 1D.
    
    Parameters
    ----------
    dx : function
        Derivative function, of form dx(x, par)
    par : unknown
        Whatever parameters the dx function needs
    x : array
        Values at which to plot the derivative
    add_arrows : array of floats
        At what locations to add arrows
    add_points : bool
        Whether to add fixed points
        
    Output
    ------
    Holoviews overlay
        Plotting object with everything necessary in it
    
    '''
    # predefine output
    out = hv.Overlay()
    
    # get derivative values
    dx_vals = dx(x, par)
    dx_sign = np.sign(np.diff(dx_vals))
    
    # get fixed points and their stability
    nulls = np.where(np.diff(np.sign(dx_vals)))[0]+1
    stable = dx_sign[nulls]==-1
    
    # plot phase 'plane'
    out *= hv.Curve(zip(x, dx_vals), vdims=dim_dr, kdims=dim_r)
    out *= hv.HLine(0)
    
    # plot optional additions
    if add_points:
        out *= hv.Scatter([]) # to fix overlay bug with HLine
        for i, point in enumerate(nulls):
            if stable[i]:
                opts = hv.opts.Scatter(color='k')
            else:
                opts = hv.opts.Scatter(color='k', fill_color='w')
            out *= hv.Scatter(zip([x[point]], [dx_vals[point]])).opts(opts)
            
            
    if add_arrows is not None:
        for loc in add_arrows:
            if dx(loc, par) > 0:
                direction = '>'
            else:
                direction = '<'
            out *= hv.Arrow(loc, 0, direction=direction)
            
    return out

def sim_animate_1D(dx, x0, par, dt, nT, downsample=1, ref_fig=None):
    '''Simlates and animates a model for given starting conditions. 
    
    Parameters
    ----------
    dx, x0, par, dt, nT
        See euler_solve() for paramter descriptions.
    downsample : int
        How many frames to downsample to
    ref_fig : HoloViews Overlay
        Reference overlay to put under animation in every frame for the x-dx/dt plot
        
    Output
    ------
    list of HoloViews objects
        List of plots to show system state in x-dx/dt plot. Can be animated with hv.HoloMap().
    list of HoloViews objects,
        Same as previous, but for t-x plot
    array
        Solved values
    '''
    # get data
    x = euler_solve(dx, x0, par, dt, nT)[0, :]
    dx_vals = dx(x, par)
    times = np.arange(nT)*dt
    
    # plot frames
    if ref_fig is None:
        ref_fig = hv.Overlay()
    frames_xdx = {f*dt: ref_fig*hv.Scatter(zip([x[f]], [dx_vals[f]])) for f in range(0, nT, downsample)}
    opts = hv.opts.Curve(color='k')
    frames_tx = {f*dt: hv.Curve(zip(times[:f], x[:f]), vdims=dim_r2, kdims=dim_t).opts(opts)*hv.Scatter(zip([times[f]], [x[f]])) for f in range(0, nT, downsample)}
    
    return x, frames_xdx, frames_tx


<center>  

# Understanding small neural networks: an introduction to dynamical system theory
<br/>
<br/>
    


<center>
Sander Keemink
    



### Dynamical systems analysis
$$ \frac{dr}{dt} = f(r, I) $$
* Models evolution over time
* Once complicated enough, usually not solvable analytically (i.e., anything in neuroscience)
* But: overall behaviour can be understood and predicted with dynamical systems theory

<center>  

# One-dimensional systems 
    
<center>

    



###  Firing rate model
$$ \frac{dr}{dt} = -r + I$$
<center><img src='figs/neural systems - simple2.svg' width=400>

In [ ]:
# Setup linear
s = 1
w = 0
I = 0.5
par = [s, w, I]

f = lambda x, s: s*x
dr = lambda x, par: -x + f(par[1]*x + par[2], par[0])

rs = np.linspace(-.5, 1.5, 1000)

# basic figures
fig = phaseplane_1D(dr, par, rs, None, False)
fig_arrows = phaseplane_1D(dr, par, rs, [0.4, 0.6], True)

# animation plus simulation
r0 = [1]
dt = 0.01
nT = 1000
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation = hv.HoloMap(frames_tx, kdims=dim_t)*hv.Curve(zip([0], [0]))
animation_full = hv.HoloMap(frames_xdx).opts(xlim=(-.5,1.5), ylim=(-1, 1)) + hv.HoloMap(frames_tx)

###  Firing rate model
$$ \frac{dr}{dt} = -r + I$$
$$ I = 0.5 $$
$$ r_0 = 1 $$

In [ ]:
animation # add I to plot or by text

####  Phase-plot representation
$$ \tau\frac{dr}{dt} = -r + I$$
$$ I = 0.5 $$

In [ ]:
fig

In [ ]:
%%opts Curve {+axiswise} Scatter {+axiswise}
animation_full

###  Fixed point analysis

In [ ]:
# make sure it's understandable what positive/negative derivatives mean
fig_arrows

###  With autapse
$$ \tau\frac{dr}{dt} = (w-1)r + I $$
<center><img src='figs/neural systems - autapse.svg' width=400>

In [ ]:
# Setup linear
s = 1
w = 0.5
I = 0.5
par = [s, w, I]

f = lambda x, s: s*x
dr = lambda x, par: -x + f(par[1]*x + par[2], par[0])

rs = np.linspace(-.5, 1.5, 1000)

# basic figures
fig = phaseplane_1D(dr, par, rs, None, False)
fig_arrows = phaseplane_1D(dr, par, rs, [0.9, 1.1], True)

# animation plus simulation
r0 = [1]
dt = 0.01
nT = 1000
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation = hv.HoloMap(frames_tx, kdims=dim_t)*hv.Curve(zip([0], [0]))
animation_full_fixed = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.2,1.5), ylim=(-0.2, 0.8)) +hv.HoloMap(frames_tx, kdims=dim_t)
r0 = [0.5]
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation_full = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.5,1.5), ylim=(-.2, .8)) +hv.HoloMap(frames_tx, kdims=dim_t)

###  With autapse
$$ \tau\frac{dr}{dt} = (w-1)r + I $$

$$ I = 0.5 $$


$$ w = 0.5 $$

In [ ]:
fig_arrows

In [ ]:
%%opts Curve {+axiswise} Scatter {+axiswise}
animation_full_fixed

In [ ]:
animation_full

In [ ]:
# Setup linear
s = 1
w = 1.5
I = -0.5
par = [s, w, I]

f = lambda x, s: s*x
dr = lambda x, par: -x + f(par[1]*x + par[2], par[0])

rs = np.linspace(-.5, 1.5, 1000)

# basic figures
fig = phaseplane_1D(dr, par, rs, None, False)
fig_arrows = phaseplane_1D(dr, par, rs, [0.7, 1.3], True)

# animation plus simulation
r0 = [1]
dt = 0.01
nT = 1000
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation = hv.HoloMap(frames_tx, kdims=dim_t)*hv.Curve(zip([0], [0]))
animation_full_fixed = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.2,1.5), ylim=(-0.2, 0.8)) +hv.HoloMap(frames_tx, kdims=dim_t)
r0 = [1.5]
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation_full = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.5,1.5), ylim=(-.2, .8)) +hv.HoloMap(frames_tx, kdims=dim_t)

###  With strong autapse
$$ \tau\frac{dr}{dt} = (w-1)r + I $$

$$ I = -0.5 $$


$$ w = 1.5 $$

In [ ]:
fig

In [ ]:
fig_arrows

In [ ]:
frames_tx[9.9]

####  Fixed point stability
<center><img src='figs/Izh_fixed-point-stability.png' width=1000>
(Izhikevich, 2010)

####  Nonlinear equations
$$ \tau\frac{dr}{dt} = -r + f(wr+I) $$

$$ f(x) = \frac{1}{1+\exp(-10x)} $$

In [ ]:
### Setup linear
s = 10
w = 1
I = -0.5
par = [s, w, I]

# functions
f = lambda x, s: 1/(1+np.exp(-s*x))
dr = lambda x, par: -x + f(par[1]*x + par[2], par[0])
x = np.linspace(-1, 1, 100)
fig_nonlin = hv.Curve(zip(x, f(x, s)), vdims='f(x)')

rs = np.linspace(-.5, 1.5, 1000)


# basic figures
fig = phaseplane_1D(dr, par, rs, None, False)
fig_arrows = phaseplane_1D(dr, par, rs, [-0.1, 0.2, 0.8, 1.1], True)

# animation plus simulation
r0 = [0.25]
dt = 0.01
nT = 1000
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation = hv.HoloMap(frames_tx, kdims=dim_t)*hv.Curve(zip([0], [0]))
animation_full_fixed = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.5,1.5), ylim=(-.5, .5)) +hv.HoloMap(frames_tx, kdims=dim_t)
r0 = [0.5]
r, frames_xdx, frames_tx = sim_animate_1D(dr, r0, par, dt, nT, downsample=10, ref_fig=fig)
animation_full = hv.HoloMap(frames_xdx, kdims=dim_t).opts(xlim=(-.5,1.5), ylim=(-.5, .5)) +hv.HoloMap(frames_tx, kdims=dim_t)

In [ ]:
fig_nonlin
# edit slide so figure doesn't fall off

### Unstable and stable fixed points in nonlinear systems

In [ ]:
fig_arrows

* Around fixed point can approximate as a linear system!

In [ ]:
animation_full_fixed

####  Question: what happens in between two stable fixed points?
<center><img src='figs/Izh_fixed-point-stability-question.png' width=800>
    
    (Izhikevich, 2010)

####  Energy landscape view
<center><img src='figs/Izh_fixed-point-energy.png' width=600>
(Izhikevich, 2010)

####  Any function can be quickly *qualitatively* understood
<center><img src='figs/Izh_any-function.png' width=600>
(Izhikevich, 2010)

####  Bifurcation: a qualitative change in behavior
<center><img src='figs/Izh_bifurcation.png' width=200>
(Izhikevich, 2010)

<center>  

# Two-dimensional systems 
    
<center>

    



###  2D systems: coupled neural populations
$$ \frac{dr_E}{dt} = -r_E + f(w_{EE}r_E - w_{EI}r_I+I) $$
$$ \frac{dr_I}{dt} = -r_I + f(w_{IE}r_E - w_{II}r_I+I) $$
$$ f(x) = \frac{1}{1+\exp(-sx)} $$
<center><img src='figs/neural systems - exc-inh pair.svg' width=600>
(often referred to as Wilson-Cowan models)

In [ ]:
# 1D phaseplane plotter
def phaseplane_2D(dx, par, x, add_vectors=True, add_clines=False, clines=None):
    '''Plots the phaseplane in 1D.
    
    Parameters
    ----------
    dx : function
        Derivative function, of form dx(x, par)
    par : unknown
        Whatever parameters the dx function needs
    x : 3D array
        Values at which to plot the derivative
    n_grid : int
        How many arrows to plot
    add_vectors : bool, true by default
        Whether to plot the vector fields
    add_clines : bool
        Whether to add the nullclines
    nullclines : list of functions, optional
        The nullclines functions, of form [f1(r), f2(r)]
    
        
    Output
    ------
    Holoviews overlay
        Plotting object with everything necessary in it
    
    '''
    # get extents
    mn = x.min()
    mx = x.max()
    xlim = (mn, mx)
    ylim = (mn, mx)
    
    # get some data vectors
    xs = np.linspace(mn, mx, 500)
    xs[xs==0] = 1e-10
    x1, x2 = x
    n_grid = x.shape[1]
    
    # predefine output
    out = hv.Overlay()
    
    # get derivative values
    dx_vals = np.array([[dx(x[:, i, j], par) for j in range(n_grid)] for i in range(n_grid)])
    
    # get angles
    angles = np.array([[np.arctan2(dx_vals[j, i, 1], dx_vals[j, i, 0]) for i in range(n_grid)] for j in range(n_grid)])

    # get lengths
    lengths = np.linalg.norm(dx_vals, axis=2)

    # plots
    fig = hv.Overlay()
    if add_vectors:
        vector_data = (x1, x2, angles, lengths)
        fig *= hv.VectorField(vector_data, kdims=[dim_re, dim_ri])
    
    if add_clines:
        cline1 = clines[0](xs)
        cline2 = clines[1](xs)
        fig*= hv.Curve(zip(cline2, xs), kdims=dim_re, vdims=dim_ri, label='rI nullcline')
        fig*= hv.Curve(zip(xs, cline1), kdims=dim_re, vdims=dim_ri, label='rE nullcline')
            
    return fig.opts(xlim=xlim, ylim=ylim)


def sim_animate_2D(dx, x0, par, dt, nT, downsample=1, ref_fig=None, extents=None):
    '''Simlates and animates a model for given starting conditions. 
    
    Parameters
    ----------
    dx, x0, par, dt, nT
        See euler_solve() for paramter descriptions.
    downsample : int
        How many frames to downsample to
    ref_fig : HoloViews Overlay
        Reference overlay to put under animation in every frame for the x1-x2 plot
    extents : 4D tuple
        Left, bottom, right, top, plotting extents
        
    Output
    ------
    list of HoloViews objects
        List of plots to show system state in x1-x2 plot. Can be animated with hv.HoloMap().
    list of HoloViews objects,
        Same as previous, but for t-x plot
    array
        Solved values
    '''
    xlim = (extents[0], extents[2])
    ylim = (extents[1], extents[3])
    
    # get data
    x = euler_solve(dx, x0, par, dt, nT)
    times = np.arange(nT)*dt
    
    # plot frames
    if ref_fig is None:
        ref_fig = hv.Overlay()
    opts = hv.opts.Curve(color='k', xlim=xlim, ylim=ylim)
    frames_x1x2 = {f: ref_fig*hv.Curve( 
                                       zip(x[0, :f], x[1, :f]), kdims=dim_re, vdims=dim_ri
                                      ).opts(opts)*hv.Scatter(zip([x[0, f]],[x[1, f]])) 
                    for f in range(0, nT, downsample)}
    
    opts1 = hv.opts.Scatter(color=colors[1])
    opts2 = hv.opts.Scatter(color=colors[0])
    frames_tx = {f: hv.Curve(
                             zip(times[:f], x[1, :f]), kdims=dim_t, vdims=dim_r, label='rI'
                             )*hv.Curve(
                                        zip(times[:f], x[0, :f]), label='rE'
                                        )*hv.Scatter(zip([times[f]], [x[0, f]])).opts(opts1)*hv.Scatter(zip([times[f]], [x[1, f]])).opts(opts2) 
                 for f in range(0, nT, downsample)}
    
    
    return x, frames_x1x2, frames_tx


In [ ]:
%%opts Curve {+axiswise}

## default: single stable state
# timescale
taus = [1, 1]

# f function
s = 5
f = lambda x, s: 1/(1+np.exp(-s*x))

# connectivity
wEE = 3
wIE = 4.4
wEI = -4.4
wII = -3
W = np.array([[wEE, wEI], [wIE, wII]])

# external input
Is = [0.3, 0.3]

# limit cycle
if False:
    taus = [0.01, 0.1]
    s = 1
    wEE = 5
    wIE = 30
    wEI = -5
    wII = -4.4
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [0.3, -10]
    
# binary state
if False: 
    taus = [1, 1]
    s = 1
    wEE = 10
    wIE = 4.4
    wEI = -4.4
    wII = -3
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [-1.8, -0.6]
    


# all inputs
par = [s, W, Is]

# function
dr2d = lambda r, par: -r/taus + f(np.dot(par[1], r) + par[2], par[0])/taus

# NULLCLINES
def null_re(re):
    x = (1-re)/re
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wEE*re + Is[0])/wEI

def null_ri(ri):
    x = (1-ri)/ri
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wII*ri + Is[1])/wIE



# setup grid
n_grid = 25
lim_grid = 1.5
r = lim_grid*np.mgrid[0:n_grid,0:n_grid]/float(n_grid) 
r1, r2 = r

# get plots
fig = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=False, clines=[null_re, null_ri])
fig_clines = phaseplane_2D(dr2d, par, r, add_vectors=False, add_clines=True, clines=[null_re, null_ri])
fig_both = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=True, clines=[null_re, null_ri])

# animate
dt = 0.01
nT = 1000
extents=(0, 0, 1.5, 1.5)
r0 = [1, 1]

r_vals, frames_x1x2, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=None, extents=extents)
r_vals, frames_x1x2_vectors, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig, extents=extents)
r_vals, frames_x1x2_both, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_both, extents=extents)
r_vals, frames_x1x2_clines, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_clines, extents=extents)

animation = hv.HoloMap(frames_x1x2, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_vectors = hv.HoloMap(frames_x1x2_vectors, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_clines = hv.HoloMap(frames_x1x2_clines, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_both = hv.HoloMap(frames_x1x2_both, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)

###  Simulation
$$ \frac{dr_E}{dt} = -r_E + f(w_{EE}r_E - w_{EI}r_I+I_E) $$
$$ \frac{dr_I}{dt} = -r_I + f(w_{IE}r_E - w_{II}r_I+I_I) $$

$$I_E = I_I = 0.3$$
$$s = 5$$
$$w_{EE} = 3$$
$$w_{IE} = 4.4$$
$$w_{EI} = -4.4$$
$$w_{II} = -3$$

In [ ]:
animation

### Phase-plane analysis

In [ ]:
animation_vectors

### Phase-plane analysis and nullclines
$$ \frac{dr_E}{dt} = -r_E + f(w_{EE}r_E - w_EIr_I+I) $$
$$ \frac{dr_I}{dt} = -r_I + f(w_{IE}r_E - w_IIr_I+I) $$

* The nullcline for a variable shows where it does not change

$$ -r_E + f(w_{EE}r_E - w_EIr_I+I) = 0$$

$$ -r_I + f(w_{IE}r_E - w_IIr_I+I) = 0$$



In [ ]:
fig_both

* Fixed points are at the intersections of nullclines

### Single fixed point


In [ ]:
animation_clines

In [ ]:
%%opts Curve {+axiswise}

## default: single stable state
# timescale
taus = [1, 1]

# f function
s = 5
f = lambda x, s: 1/(1+np.exp(-s*x))

# connectivity
wEE = 3
wIE = 4.4
wEI = -4.4
wII = -3
W = np.array([[wEE, wEI], [wIE, wII]])

# external input
Is = [0.3, 0.3]

# limit cycle
if False:
    taus = [0.01, 0.1]
    s = 1
    wEE = 5
    wIE = 30
    wEI = -5
    wII = -4.4
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [0.3, -10]
    
# binary state
if True: 
    taus = [1, 1]
    s = 1
    wEE = 10
    wIE = 4.4
    wEI = -4.4
    wII = -3
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [-1.8, -0.6]
    
# all inputs
par = [s, W, Is]

# function
dr2d = lambda r, par: -r/taus + f(np.dot(par[1], r) + par[2], par[0])/taus

# NULLCLINES
def null_re(re):
    x = (1-re)/re
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wEE*re + Is[0])/wEI

def null_ri(ri):
    x = (1-ri)/ri
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wII*ri + Is[1])/wIE



# setup grid
n_grid = 25
lim_grid = 1.5
r = lim_grid*np.mgrid[0:n_grid,0:n_grid]/float(n_grid) 
r1, r2 = r

# get plots
fig = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=False, clines=[null_re, null_ri])
fig_clines = phaseplane_2D(dr2d, par, r, add_vectors=False, add_clines=True, clines=[null_re, null_ri])
fig_both = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=True, clines=[null_re, null_ri])

# animate
dt = 0.01
nT = 2000
extents=(0, 0, 1.5, 1.5)
r0 = [.01, .3]
r_vals, frames_x1x2, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=None, extents=extents)
r_vals, frames_x1x2_vectors, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig, extents=extents)
r_vals, frames_x1x2_both, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_both, extents=extents)
r_vals, frames_x1x2_clines, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_clines, extents=extents)

animation = hv.HoloMap(frames_x1x2, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_vectors = hv.HoloMap(frames_x1x2_vectors, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_clines = hv.HoloMap(frames_x1x2_clines, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_both = hv.HoloMap(frames_x1x2_both, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)

### 'Ghost' or 'Ruin' of an attractor



In [ ]:
animation_both

In [ ]:
%%opts Curve {+axiswise}
   
# binary state
if True: 
    taus = [1, 1]
    s = 1
    wEE = 10
    wIE = 4.4
    wEI = -4.4
    wII = -3
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [-1.8, 0.6]
    


# all inputs
par = [s, W, Is]

# function
dr2d = lambda r, par: -r/taus + f(np.dot(par[1], r) + par[2], par[0])/taus

# NULLCLINES
def null_re(re):
    x = (1-re)/re
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wEE*re + Is[0])/wEI

def null_ri(ri):
    x = (1-ri)/ri
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wII*ri + Is[1])/wIE



# setup grid
n_grid = 25
lim_grid = 1.5
r = lim_grid*np.mgrid[0:n_grid,0:n_grid]/float(n_grid) 
r1, r2 = r

# get plots
fig = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=False, clines=[null_re, null_ri])
fig_clines = phaseplane_2D(dr2d, par, r, add_vectors=False, add_clines=True, clines=[null_re, null_ri])
fig_both = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=True, clines=[null_re, null_ri])

# animate
dt = 0.01
nT = 1000
extents=(0, 0, 1.5, 1.5)
r0 = [.4, .3]
r_vals, frames_x1x2, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=None, extents=extents)
r_vals, frames_x1x2_vectors, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig, extents=extents)
r_vals, frames_x1x2_both, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_both, extents=extents)
r_vals, frames_x1x2_clines, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_clines, extents=extents)


animation = hv.HoloMap(frames_x1x2, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_vectors = hv.HoloMap(frames_x1x2_vectors, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_clines = hv.HoloMap(frames_x1x2_clines, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_both = hv.HoloMap(frames_x1x2_both, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)

### Several fixed points


In [ ]:
animation_both

### Limit cycles

In [ ]:
%%opts Curve {+axiswise}

## default: single stable state
# timescale
taus = [0.01, 0.01]

# f function
s = 5
f = lambda x, s: 1/(1+np.exp(-s*x))

# connectivity
wEE = 3
wIE = 4.4
wEI = -4.4
wII = -3
W = np.array([[wEE, wEI], [wIE, wII]])

# external input
Is = [0.3, 0.3]

# limit cycle
if True:
    taus = [0.01, 0.1]
    s = 1
    wEE = 5
    wIE = 30
    wEI = -5
    wII = -4.4
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [0.3, -10]
    
# binary state
if False: 
    taus = [0.01, 0.01]
    s = 1
    wEE = 10
    wIE = 4.4
    wEI = -4.4
    wII = -3
    W = np.array([[wEE, wEI], [wIE, wII]])
    Is = [-1.8, -0.6]
    


# all inputs
par = [s, W, Is]

# function
dr2d = lambda r, par: -r/taus + f(np.dot(par[1], r) + par[2], par[0])/taus

# NULLCLINES
def null_re(re):
    x = (1-re)/re
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wEE*re + Is[0])/wEI

def null_ri(ri):
    x = (1-ri)/ri
    x[x<=0]=1e-100
    
    return -(np.log(x)/s + wII*ri + Is[1])/wIE



# setup grid
n_grid = 25
lim_grid = 1.5
r = lim_grid*np.mgrid[0:n_grid,0:n_grid]/float(n_grid) 
r1, r2 = r

# get plots
fig = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=False, clines=[null_re, null_ri])
fig_clines = phaseplane_2D(dr2d, par, r, add_vectors=False, add_clines=True, clines=[null_re, null_ri])
fig_both = phaseplane_2D(dr2d, par, r, add_vectors=True, add_clines=True, clines=[null_re, null_ri])

# animate
dt = 0.001
nT = 1000
extents=(0, 0, 1.5, 1.5)
r0 = [1, 1]
r_vals, frames_x1x2, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=None, extents=extents)
r_vals, frames_x1x2_vectors, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig, extents=extents)
r_vals, frames_x1x2_both, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_both, extents=extents)
r_vals, frames_x1x2_clines, frames_tx = sim_animate_2D(dr2d, r0, par, dt, nT, downsample=10, ref_fig=fig_clines, extents=extents)


animation = hv.HoloMap(frames_x1x2, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_vectors = hv.HoloMap(frames_x1x2_vectors, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_clines = hv.HoloMap(frames_x1x2_clines, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)
animation_both = hv.HoloMap(frames_x1x2_both, kdims=dim_t)+hv.HoloMap(frames_tx, kdims=dim_t)

In [ ]:
animation_both

### Analyzing fixed points analytically

<center><img src='figs/Izh_fixed-point-linearized.png' width=800>
    


<center><img src='figs/Izh_fixed-point-eigenvectors.png' width=800>
    


### Real eigenvalues, negative or positive

<center><img src='figs/Izh_fixed_point_unstable_stable.png' width=600>
    (Izhikevich, 2010)

### Real eigenvalues, mix of positive and negative

<center><img src='figs/Izh_saddle.png' width=600>
    (Izhikevich, 2010)

### Complex eigenvalues

<center><img src='figs/Izh_focus.png' width=600>
    (Izhikevich, 2010)

### Relationship between eigenvalues and fixed point types

<center><img src='figs/Izh_fixed-points-types.png' width=600>
    


### Saddle-node bifurcation

<center><img src='figs/Izh_saddle-node.png' width=600>
    (Izhikevich, 2010)

*And many others, see further reading!

### Example applications

### Understanding microcircuits
<center><img src='figs/V1MicroCircuitModel.png' width=1200>
    
(Garcia del Molino et al., 2017)

### Understanding large neural networks
<center><img src='figs/NetworkReduction.png' width=300>
    
(Wong & Wang, 2006)

<center><img src='figs/NetworkReductionPhasePlane.png', width=900>
    
(Wong & Wang, 2006)

### Dynamical systems as an energy landscape
<center><img src='figs/EnergyLandscapes.png' width=900>
    
( https://commons.wikimedia.org/wiki/File:Energy_landscape.png,
https://en.wikipedia.org/wiki/Folding_funnel,
https://www.pathwayz.org/Tree/Plain/ACTIVATION+ENERGY+%5BENZYMES%5D)

### Summary
* Even simple neural interactions can be complicated and hard to solve
* Phase-plane analysis gives powerful intuitions
* Important to at least intuitively understand dynamics for all of biology

### Further info
Online pages (scholarpedia, wikipedia)
* https://en.wikipedia.org/wiki/Phase_plane
* http://www.scholarpedia.org/article/State_space
* https://en.wikipedia.org/wiki/Attractor

Online courses
* Coursera - Computational Neuroscience, two lectures in week 5

Books
* Dynamical systems in neuroscience by Izhikevich
* Nonlinear Dynamics and Chaos by Strogatz


<center>

# Questions?

swkeemink@scimail.eu
